<a href="https://colab.research.google.com/github/PianoPlayer09/ConvLSTM/blob/main/NewApproach_Paper_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
ee.Authenticate()

In [ ]:
import ee
import geemap

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

year, month = 2022, 9
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Functions
# ========================================
def mask_clouds(img):
    scl = img.select('SCL')
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

def add_indices(img):
    bands = ['B2','B3','B4','B8','B11','B12']
    scaled = img.select(bands).multiply(0.0001)
    img = img.addBands(scaled, overwrite=True)

    ndsi = img.normalizedDifference(['B3','B11']).rename('NDSI')
    ndwi = img.normalizedDifference(['B3','B8']).rename('NDWI')
    awei = img.expression('4*G - 0.25*N + 2.75*S',
                          {'G': img.select('B3'),
                           'N': img.select('B8'),
                           'S': img.select('B11')}).rename('AWEI')
    return img.addBands([ndsi, ndwi, awei])

# ========================================
# Get composite
# ========================================
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start, end)
        .map(mask_clouds)
        .map(add_indices))

comp = s2.median()

# ========================================
# Create classification layers
# ========================================
ndsi = comp.select('NDSI')
ndwi = comp.select('NDWI')
awei = comp.select('AWEI')

# Define classes
snow = ndsi.gt(0.4).rename('snow')                          # 1 = snow
water_ndwi = ndwi.gt(0.2).And(snow.Not()).rename('water')  # 1 = water (not snow)
water_awei = awei.gt(0).And(snow.Not()).rename('water_awei')

# Combined classification
classification = ee.Image(0).where(snow, 1).where(water_ndwi, 2).where(water_awei, 3)

# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map(center=[70, -45], zoom=12)

# True color
Map.addLayer(comp.clip(geom),
             {'bands':['B4','B3','B2'], 'min':0, 'max':0.3},
             'True Color')

# NDSI
Map.addLayer(ndsi.clip(geom),
             {'min':-0.5, 'max':1, 'palette':['blue','white','red']},
             'NDSI')

# NDWI
Map.addLayer(ndwi.clip(geom),
             {'min':-0.5, 'max':1, 'palette':['brown','white','blue']},
             'NDWI')

# AWEI
Map.addLayer(awei.clip(geom),
             {'min':-1, 'max':1, 'palette':['red','white','blue']},
             'AWEI')

# Classification (THE MONEY SHOT)
Map.addLayer(classification.clip(geom),
             {'min':0, 'max':3, 'palette':['black','white','blue','cyan']},
             'Classification (0=other, 1=snow, 2=water_NDWI, 3=water_AWEI)')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary')
Map.centerObject(geom, 13)

Map

Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
# Count pixels in each class
stats = classification.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=geom,
    scale=10,
    maxPixels=int(1e9)
).getInfo()

total = sum(stats['constant'].values())
print("\nPixel Classification:")
for cls, count in stats['constant'].items():
    labels = {0:'Other', 1:'Snow', 2:'Water(NDWI)', 3:'Water(AWEI)'}
    print(f"  {labels.get(int(cls), cls)}: {count} px ({100*count/total:.1f}%)")


Pixel Classification:
  Snow: 1183.749019607842 px (100.0%)


In [ ]:
# ========================================
# Create classification layers
# ========================================
ndsi = comp.select('NDSI') # (Green - SWIR) -> High for Snow, High for Water
nir  = comp.select('B8')   # NIR Band -> High for Snow, Low for Water

# --- THIS IS THE NEW LOGIC ---

# 1. Create a "candidate" mask of everything that could be snow OR water.
#    We use NDSI > 0.4 (a common threshold)
is_snow_or_water = ndsi.gt(0.4)

# 2. Separate snow from water using the fact that water absorbs NIR
#    and snow reflects it.
#    We use a threshold on the scaled NIR band ('B8').
#    Water has LOW NIR reflectance (e.g., < 0.15)
#    Snow has HIGH NIR reflectance (e.g., > 0.15)
#    You can adjust this threshold if needed!
nir_threshold = 0.15

# FINAL CLASSES
# Snow = 1: It's a candidate AND its NIR reflectance is HIGH
snow = is_snow_or_water.And(nir.gt(nir_threshold)).rename('snow')

# Water = 2: It's a candidate AND its NIR reflectance is LOW
water = is_snow_or_water.And(nir.lt(nir_threshold)).rename('water')

# Combined classification
# Start with 0 (other)
classification = ee.Image(0).rename('classification')
# Burn in water as 2
classification = classification.where(water, 2)
# Burn in snow as 1 (this overwrites any 'other' pixels)
classification = classification.where(snow, 1)

# --- END OF NEW LOGIC ---


# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map(center=[70, -45], zoom=12)

# True color
Map.addLayer(comp.clip(geom),
               {'bands':['B4','B3','B2'], 'min':0, 'max':0.3},
               'True Color')

# NDSI (The "problem" index)
Map.addLayer(ndsi.clip(geom),
               {'min':-0.5, 'max':1, 'palette':['#0000AA','white','#AA0000']},
               'NDSI (High = Snow OR Water)')

# NIR (The "tie-breaker" band)
Map.addLayer(nir.clip(geom),
               {'min':0, 'max':0.5, 'palette':['black','white']},
               'NIR (B8) (Dark = Water, Bright = Snow)')

# Classification (THE MONEY SHOT)
Map.addLayer(classification.clip(geom).selfMask(), # .selfMask() makes 0 transparent
               {'min':1, 'max':2, 'palette':['white','blue']},
               'Classification (1=snow, 2=water)')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary')
Map.centerObject(geom, 13)

Map

Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display  # <-- CHANGE 1: Import the display function

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

year, month = 2022, 10
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Get images WITHOUT cloud mask first
# ========================================
s2_raw = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(geom)
           .filterDate(start, end))

print(f"Found {s2_raw.size().getInfo()} raw images")

if s2_raw.size().getInfo() > 0:
    # Take first image
    img = s2_raw.first()

    Map = geemap.Map(center=[70, -45], zoom=12)

    # ========================================
    # 1. TRUE COLOR (what human eye sees)
    # ========================================
    Map.addLayer(img.clip(geom),
                 {'bands':['B4','B3','B2'], 'min':0, 'max':3000},
                 'True Color (raw DN)')

    # ========================================
    # 2. FALSE COLOR (NIR shows vegetation/water)
    # ========================================
    Map.addLayer(img.clip(geom),
                 {'bands':['B8','B4','B3'], 'min':0, 'max':3000},
                 'False Color (NIR-R-G)')

    # ========================================
    # 3. SCL (Scene Classification - from Sentinel)
    # ========================================
    Map.addLayer(img.select('SCL').clip(geom),
                 {'min':0, 'max':11, 'palette':['000000','ff0000','gray','brown',
                                                'green','yellow','blue','purple',
                                                'white','cyan','pink','orange']},
                 'SCL Classes')

    # ========================================
    # 4. NDSI (raw, no scaling)
    # ========================================
    ndsi_raw = img.normalizedDifference(['B3','B11'])
    Map.addLayer(ndsi_raw.clip(geom),
                 {'min':-1, 'max':1, 'palette':['blue','white','red']},
                 'NDSI (unscaled)')

    # ========================================
    # 5. Now WITH scaling
    # ========================================
    scaled = img.select(['B3','B8','B11']).multiply(0.0001)
    ndsi_scaled = scaled.normalizedDifference(['B3','B11'])
    ndwi_scaled = scaled.normalizedDifference(['B3','B8'])

    Map.addLayer(ndsi_scaled.clip(geom),
                 {'min':-0.5, 'max':1, 'palette':['blue','white','red']},
                 'NDSI (scaled)')

    Map.addLayer(ndwi_scaled.clip(geom),
                 {'min':-0.5, 'max':1, 'palette':['brown','white','blue']},
                 'NDWI (scaled)')

    Map.addLayer(geom, {'color':'yellow'}, 'Lake boundary')
    Map.centerObject(geom, 13)

    # ========================================
    # Print metadata
    # ========================================
    props = img.toDictionary().getInfo()
    print("\nImage metadata:")
    print(f"   Date: {props.get('system:index', 'unknown')}")
    print(f"   Cloud %: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'unknown')}")

    display(Map)  # <-- CHANGE 2: Explicitly call display() on the Map

else:
    print("❌ No images found - try different date range")

Found 18 raw images

Image metadata:
   Date: unknown
   Cloud %: 82.182324


Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

year, month = 2022, 8
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Get images
# ========================================
s2_raw = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(geom)
           .filterDate(start, end))

print(f"Found {s2_raw.size().getInfo()} raw images")

if s2_raw.size().getInfo() > 0:
    # Take first image
    img = s2_raw.first()

    # ========================================
    # *** NEW: Create a SPLIT MAP ***
    # ========================================
    Map = geemap.SplitMap(center=[70, -45], zoom=12)
    # Map.add_basemap("SATELLITE") # Optional: add a satellite base

    # ========================================
    # 1. TRUE COLOR (The "raw" image) -> GOES ON THE LEFT
    # ========================================
    Map.addLayer_left(img.clip(geom),
                 {'bands':['B4','B3','B2'], 'min':0, 'max':3000},
                 'True Color (raw DN)')

    # ========================================
    # 2. FALSE COLOR -> GOES ON THE RIGHT
    # ========================================
    Map.addLayer_right(img.clip(geom),
                 {'bands':['B8','B4','B3'], 'min':0, 'max':3000},
                 'False Color (NIR-R-G)')

    # ========================================
    # 3. SCL (Classification) -> GOES ON THE RIGHT
    # ========================================
    Map.addLayer_right(img.select('SCL').clip(geom),
                 {'min':0, 'max':11, 'palette':['000000','ff0000','gray','brown',
                                                'green','yellow','blue','purple',
                                                'white','cyan','pink','orange']},
                 'SCL Classes')

    # ========================================
    # 4. NDSI (raw, no scaling) -> GOES ON THE RIGHT
    # ========================================
    ndsi_raw = img.normalizedDifference(['B3','B11'])
    Map.addLayer_right(ndsi_raw.clip(geom),
                 {'min':-1, 'max':1, 'palette':['blue','white','red']},
                 'NDSI (unscaled)')

    # ========================================
    # 5. Scaled Indices -> GOES ON THE RIGHT
    # ========================================
    scaled = img.select(['B3','B8','B11']).multiply(0.0001)
    ndsi_scaled = scaled.normalizedDifference(['B3','B11'])
    ndwi_scaled = scaled.normalizedDifference(['B3','B8'])

    Map.addLayer_right(ndsi_scaled.clip(geom),
                 {'min':-0.5, 'max':1, 'palette':['blue','white','red']},
                 'NDSI (scaled)')

    Map.addLayer_right(ndwi_scaled.clip(geom),
                 {'min':-0.5, 'max':1, 'palette':['brown','white','blue']},
                 'NDWI (scaled)')

    # Add the lake boundary to BOTH sides so it lines up
    Map.addLayer_left(geom, {'color':'yellow'}, 'Lake boundary')
    Map.addLayer_right(geom, {'color':'yellow'}, 'Lake boundary')
    Map.centerObject(geom, 13)

    # ========================================
    # Print metadata
    # ========================================
    props = img.toDictionary().getInfo()
    print("\nImage metadata:")
    print(f"   Date: {props.get('system:index', 'unknown')}")
    print(f"   Cloud %: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'unknown')}")

    display(Map)  # This displays the split map

else:
    print("❌ No images found - try different date range")

Found 19 raw images


AttributeError: module 'geemap' has no attribute 'SplitMap'

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

year, month = 2022, 8
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Get images
# ========================================
s2_raw = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(geom)
           .filterDate(start, end))

print(f"Found {s2_raw.size().getInfo()} raw images")

if s2_raw.size().getInfo() > 0:
    # Take the first available image
    img = s2_raw.first()

    # ========================================
    # Create a SIMPLE, NORMAL map
    # ========================================
    Map = geemap.Map(center=[70, -45], zoom=12)

    # ========================================
    # Add ONLY the "raw camera image"
    # This uses the Red(B4), Green(B3), and Blue(B2) bands
    # ========================================
    Map.addLayer(img.clip(geom),
                 {'bands':['B4','B3','B2'], 'min':0, 'max':3000},
                 'True Color (The "Raw" Image)')

    # Add the lake boundary so we can find it
    Map.addLayer(geom, {'color':'yellow'}, 'Lake boundary')
    Map.centerObject(geom, 13)

    # ========================================
    # Print metadata
    # ========================================
    props = img.toDictionary().getInfo()
    print("\nImage metadata:")
    print(f"   Date: {props.get('system:index', 'unknown')}")
    print(f"   Cloud %: {props.get('CLOUDY_PIXEL_PERCENTAGE', 'unknown')}")

    # Explicitly display the simple map
    display(Map)

else:
    print("❌ No images found - try different date range")

Found 19 raw images

Image metadata:
   Date: unknown
   Cloud %: 23.666243


Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

year, month = 2017, 8
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# 1. Get Sentinel-2 Composite (for visual reference)
# ========================================

def mask_clouds(img):
    scl = img.select('SCL')
    # Masks 3(cloud shadow), 8(cloud med prob), 9(cloud high prob), 10(cirrus)
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
       .filterBounds(geom)
       .filterDate(start, end)
       .map(mask_clouds))

# Create a true-color composite
comp = s2.median()
vis_true_color = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# ========================================
# 2. Get Dynamic World Classification
# ========================================

# Load the Dynamic World collection
dw_collection = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
                   .filterBounds(geom)
                   .filterDate(start, end))

# Create a median composite of the probability bands
dw_composite = dw_collection.median()

# Select the probability bands for water and snow
water_prob = dw_composite.select('water')
snow_prob = dw_composite.select('snow_and_ice')

# Create a classification:
# 1 = Water (where water prob > 50%)
# 2 = Snow (where snow prob > 50%)
# 0 = Other
threshold = 0.5
is_water = water_prob.gt(threshold)
is_snow = snow_prob.gt(threshold)

# Start with a base image of 0 (Other)
classification = ee.Image(0).rename('classification')
# Burn in water as 1, then snow as 2 (order matters if they overlap)
classification = classification.where(is_water, 1).where(is_snow, 2)


# ========================================
# 3. Calculate Statistics
# ========================================
print("Calculating Dynamic World statistics...")
stats = classification.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=geom,
    scale=10,  # Dynamic World is 10m scale
    maxPixels=1e9
).getInfo()

class_counts = stats.get('classification')

if not class_counts:
    print("No classified pixels found in the geometry.")
else:
    labels = {
        0: 'Other',
        1: 'Water',
        2: 'Snow / Ice'
    }
    total_pixels = sum(class_counts.values())
    print("\n--- Dynamic World Classification Stats ---")
    print(f"Total Classified Pixels: {total_pixels}")

    for class_id_str, count in class_counts.items():
        class_id_int = int(class_id_str)
        label = labels.get(class_id_int, "Unknown")
        percentage = (count / total_pixels) * 100
        print(f"  {label}: {count} pixels ({percentage:.2f}%)")


# ========================================
# 4. VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 12)

# Add the "real" image for comparison
Map.addLayer(comp.clip(geom), vis_true_color, 'True Color (Sentinel-2)')

# Add the Dynamic World classification
classification_palette = ['black', 'blue', 'white'] # 0=black, 1=blue, 2=white
Map.addLayer(classification.clip(geom).selfMask(),
               {'min': 1, 'max': 2, 'palette': classification_palette},
               'Dynamic World (1=Water, 2=Snow)')

Map.addLayer(geom, {'color': 'red'}, 'Lake boundary', True, 0.5)
display(Map)

Calculating Dynamic World statistics...

--- Dynamic World Classification Stats ---
Total Classified Pixels: 1183.7490196078431
  Other: 233.6745098039216 pixels (19.74%)
  Water: 661.5333333333334 pixels (55.88%)
  Snow / Ice: 288.54117647058814 pixels (24.38%)


Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

# --- Setup from your previous code ---
ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geometry = lake.geometry() # Use your 'geometry'

# Define start and end dates
year, month = 2022, 9
startDate = ee.Date.fromYMD(year, month, 1)
endDate = startDate.advance(1, 'month')

# ========================================
# --- Your JavaScript code, translated to Python ---
# ========================================

# 1. Define the Sentinel-2 image collection
s2 = (ee.ImageCollection('COPERNICUS/S2_HARMONIZED')
        .filterDate(startDate, endDate)
        .filterBounds(geometry)
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 35)))

# 2. Get the first image from the collection
s2Image = ee.Image(s2.first())

# 3. Define the visualization parameters
s2VisParams = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

# 4. Create a map and add the layer
Map = geemap.Map()
Map.centerObject(geometry, 13)
Map.addLayer(s2Image, s2VisParams, 'Sentinel-2 Image')
# Add this line right before display(Map)
Map.addLayer(geometry, {'color': 'yellow'}, 'Lake Boundary')
# 5. Display the map
display(Map)

Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

# --- Assuming 'classification' and 'geom' exist from the previous cell ---

# ========================================
# 1. Create the new "merged" classification
# ========================================

# We will re-classify:
# 1 = Water
# 2 = Snow / Ice

# Select all pixels that were snow (class 2)
is_snow = classification.eq(2)

# Create a new image.
# Start by making EVERYTHING '1' (Water).
# Then, use .where() to put '2' (Snow) back on top.
merged_classification = ee.Image(1).rename('classification_merged') \
                          .where(is_snow, 2)


# ========================================
# 2. Recalculate Statistics for the new image
# ========================================
print("Calculating merged classification statistics...")
merged_stats = merged_classification.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=geom,
    scale=10,
    maxPixels=1e9
).getInfo()

class_counts_merged = merged_stats.get('classification_merged')

if not class_counts_merged:
    print("No classified pixels found in the geometry.")
else:
    # Define our new, simpler labels
    labels = {
        1: 'Water (Merged)',
        2: 'Snow / Ice'
    }
    total_pixels = sum(class_counts_merged.values())
    print("\n--- Merged Classification Stats ---")
    print(f"Total Classified Pixels: {total_pixels}")

    for class_id_str, count in class_counts_merged.items():
        class_id_int = int(class_id_str)
        label = labels.get(class_id_int, "Unknown")
        percentage = (count / total_pixels) * 100
        print(f"  {label}: {count} pixels ({percentage:.2f}%)")


# ========================================
# 3. VISUALIZE! 🎨
# ========================================
Map_merged = geemap.Map()
Map_merged.centerObject(geom, 12)

# Add the "real" image for comparison (assuming 'comp' is still in memory)
try:
    vis_true_color = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
    Map_merged.addLayer(comp.clip(geom), vis_true_color, 'True Color (Sentinel-2)')
except NameError:
    print("Run the previous cell to load the 'comp' image for comparison.")

# Add the new merged Dynamic World classification
# Our palette is now just [blue, white] for classes [1, 2]
merged_palette = ['blue', 'white']
Map_merged.addLayer(merged_classification.clip(geom).selfMask(),
                   {'min': 1, 'max': 2, 'palette': merged_palette},
                   'Merged Classification (1=Water, 2=Snow)')

Map_merged.addLayer(geom, {'color': 'red'}, 'Lake boundary', True, 0.5)
display(Map_merged)

Calculating merged classification statistics...

--- Merged Classification Stats ---
Total Classified Pixels: 1183.7490196078431
  Water (Merged): 895.2078431372549 pixels (75.62%)
  Snow / Ice: 288.54117647058814 pixels (24.38%)


Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

# --- Setup from your previous code ---
ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry() # Your lake boundary

# Define start and end dates
year, month = 2020, 9
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# 1. Get the Dynamic World "label" Composite
# ========================================
dw_collection = (ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1')
                   .filterBounds(geom)
                   .filterDate(start, end))

# Select the 'label' band and get the *mode* (most common class)
dw_labels_composite = dw_collection.select('label').mode()

# ========================================
# 2. Define Visualization for 9 Classes
# ========================================
# These are the official colors and labels (from your helper code)
dw_vis_params = {
  'min': 0, 'max': 8,
  'palette': [
    '#419BDF', # 0 = Water
    '#397D49', # 1 = Trees
    '#88B053', # 2 = Grass
    '#7A87C6', # 3 = Flooded Vegetation
    '#E49635', # 4 = Crops
    '#DFC35A', # 5 = Shrub & Scrub
    '#C4281B', # 6 = Built
    '#A59B8F', # 7 = Bare
    '#B39FE1'  # 8 = Snow & Ice
  ]
}

# Labels for printing stats
class_labels = {
    0: 'Water',
    1: 'Trees',
    2: 'Grass',
    3: 'Flooded Vegetation',
    4: 'Crops',
    5: 'Shrub & Scrub',
    6: 'Built',
    7: 'Bare',
    8: 'Snow & Ice'
}

# ========================================
# 3. Calculate Statistics for ALL classes
# ========================================
print("Calculating full classification statistics...")
stats = dw_labels_composite.reduceRegion(
    reducer=ee.Reducer.frequencyHistogram(),
    geometry=geom,
    scale=10,
    maxPixels=1e9
).getInfo()

# The band is named 'label'
class_counts = stats.get('label')

if not class_counts:
    print("No classified pixels found in the geometry.")
else:
    total_pixels = sum(class_counts.values())
    print("\n--- Full Dynamic World Classification Stats ---")
    print(f"Total Classified Pixels: {total_pixels}")

    for class_id_str, count in class_counts.items():
        class_id_int = int(class_id_str)
        # Use our dictionary to get the label name
        label = class_labels.get(class_id_int, "Unknown")
        percentage = (count / total_pixels) * 100
        print(f"  {label} (Class {class_id_int}): {count} pixels ({percentage:.2f}%)")

# ========================================
# 4. VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 12)

# Add the "real" image for comparison
try:
    s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterBounds(geom).filterDate(start, end).median())
    Map.addLayer(s2.clip(geom),
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000},
                   'True Color (Sentinel-2)')
except Exception as e:
    print(f"Could not load Sentinel-2 image for comparison: {e}")


# Add the full Dynamic World classification
Map.addLayer(dw_labels_composite.clip(geom),
               dw_vis_params,
               'Dynamic World (All Classes)')

# Add the lake boundary
Map.addLayer(geom, {'color': 'red'}, 'Lake boundary', True, 0.5)

# --- THIS IS THE FIX ---
# Use 'Dynamic_World' with an underscore
Map.add_legend(builtin_legend='Dynamic_World', title='Dynamic World LULC')

display(Map)

Calculating full classification statistics...

--- Full Dynamic World Classification Stats ---
Total Classified Pixels: 1183.7490196078434
  Water (Class 0): 950.9490196078433 pixels (80.33%)
  Snow & Ice (Class 8): 232.8 pixels (19.67%)


Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

So we are doing Dynamic World above, below lets try this other thing AWEI

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

# Using your date where you are confident the data is good
year, month = 2020, 9
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Functions
# ========================================
def mask_clouds(img):
    """Masks clouds and cloud shadows in Sentinel-2 SCL."""
    scl = img.select('SCL')
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

def add_korinek_indices(img):
    """Calculates the two indices needed for the Korinek method:
    1. NDWI (Green/NIR) to find water.
    2. NDSI (Green/SWIR) to find snow/ice.
    """

    bands = ['B3','B4','B8','B11'] # B4 for True Color
    scaled = img.select(bands).multiply(0.0001)
    img = img.addBands(scaled, overwrite=True)

    # 1. NDWI (Green/NIR) - This is good at finding WATER, bad at snow
    ndwi = img.normalizedDifference(['B3','B8']).rename('NDWI')

    # 2. NDSI (Green/SWIR) - This finds BOTH water and snow
    ndsi = img.normalizedDifference(['B3','B11']).rename('NDSI')

    return img.addBands([ndwi, ndsi])

# ========================================
# Get composite
# ========================================
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start, end)
        .map(mask_clouds)
        .map(add_korinek_indices))

comp = s2.median()

# ========================================
# Create the Korinek Hybrid Classification
# ========================================
ndwi = comp.select('NDWI') # Green/NIR
ndsi = comp.select('NDSI') # Green/SWIR

# Class 2: WATER
# Find pixels that look like water using the Green/NIR index
is_water = ndwi.gt(0.2)

# Class 1: SNOW/ICE
# Find pixels that look like snow using the Green/SWIR index...
is_snow = ndsi.gt(0.4)
# ...BUT, they must NOT look like water (using the Green/NIR index)
is_true_snow = is_snow.And(is_water.Not())

# Combine into a final map
# 0=other, 1=snow, 2=water
classification = ee.Image(0).where(is_water, 2).where(is_true_snow, 1)

# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 13)

# True color (for comparison)
Map.addLayer(comp.clip(geom),
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3},
             'True Color')

# --- THIS IS THE KORINEK CLASSIFICATION ---
# 0=transparent, 1=white (snow), 2=blue (water)
Map.addLayer(classification.clip(geom).selfMask(),
             {'min': 1, 'max': 2, 'palette':['white', 'blue']},
             'Korinek Classification (Ice=White, Water=Blue)')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary', True, 0.5)
display(Map)

Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

# Using your date
year, month = 2020, 8
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Functions
# ========================================
def mask_clouds(img):
    """Masks clouds and cloud shadows in Sentinel-2 SCL."""
    scl = img.select('SCL')
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

def add_mndwi_nir(img):
    """Calculates mNDWI (aka NDSI) and adds the NIR band (B8)."""

    bands = ['B3','B4','B8','B11'] # Green, Red, NIR, SWIR
    scaled = img.select(bands).multiply(0.0001)
    img = img.addBands(scaled, overwrite=True)

    # 1. mNDWI (aka NDSI) - This finds BOTH water and snow
    mndwi = img.normalizedDifference(['B3','B11']).rename('mNDWI')

    # 2. Get the NIR band (B8)
    nir = img.select('B8').rename('NIR')

    return img.addBands([mndwi, nir])

# ========================================
# Get composite
# ========================================
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start, end)
        .map(mask_clouds)
        .map(add_mndwi_nir))

comp = s2.median()

# ========================================
# Create the "mNDWI + NIR" Classification
# ========================================
mndwi = comp.select('mNDWI')
nir = comp.select('NIR')

# Set thresholds
MNDWI_THRESH = 0.4 # Finds all bright pixels (snow & turbid water)
NIR_THRESH = 0.12  # The tie-breaker (water is < 0.12, snow is > 0.12)

# 1. Find all pixels that are either snow or water
is_snow_or_water = mndwi.gt(MNDWI_THRESH)

# 2. Class 2: WATER
# These are pixels from above that are DARK in the NIR band
is_water = is_snow_or_water.And(nir.lt(NIR_THRESH))

# 3. Class 1: SNOW/ICE
# These are pixels from above that are BRIGHT in the NIR band
is_ice = is_snow_or_water.And(nir.gte(NIR_THRESH))

# Combine into a final map
# 0=other, 1=snow, 2=water
classification = ee.Image(0).where(is_water, 2).where(is_ice, 1)

# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 13)

# True color (for comparison)
Map.addLayer(comp.clip(geom),
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3},
             'True Color (Bluish)')

# --- THIS IS THE FINAL CLASSIFICATION ---
# 0=transparent, 1=white (snow), 2=blue (water)
Map.addLayer(classification.clip(geom).selfMask(),
             {'min': 1, 'max': 2, 'palette':['white', 'blue']},
             'mNDWI + NIR (Ice=White, Water=Blue)')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary', True, 0.5)
display(Map)

Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

# Using your date
year, month = 2020, 8
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Functions
# ========================================
def mask_clouds(img):
    """Masks clouds and cloud shadows in Sentinel-2 SCL."""
    scl = img.select('SCL')
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

def add_awei_ndsi(img):
    """Calculates AWEI (for water) and NDSI (for ice)."""

    bands = ['B2','B3','B4','B8','B11','B12']
    scaled = img.select(bands).multiply(0.0001)
    img = img.addBands(scaled, overwrite=True)

    # 1. AWEI (to find dark water)
    awei = img.expression(
        'B2 + 2.5*B3 - 1.5*(B8 + B11) - 0.25*B12',
        {'B2': img.select('B2'), 'B3': img.select('B3'),
         'B8': img.select('B8'), 'B11': img.select('B11'), 'B12': img.select('B12')}
    ).rename('AWEI')

    # 2. NDSI (to find snow/ice)
    ndsi = img.normalizedDifference(['B3','B11']).rename('NDSI')

    return img.addBands([awei, ndsi])

# ========================================
# Get composite
# ========================================
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start, end)
        .map(mask_clouds)
        .map(add_awei_ndsi))

comp = s2.median()

# ========================================
# Create the AWEI + NDSI Hybrid Classification
# ========================================
awei = comp.select('AWEI')
ndsi = comp.select('NDSI')

# Set thresholds
AWEI_THRESH = 0
NDSI_THRESH = 0.4

# 1. Class 1: SNOW/ICE
# Find all pixels that look like snow
is_ice = ndsi.gt(NDSI_THRESH)

# 2. Class 2: WATER
# Find pixels that look like water...
is_awei_water = awei.gt(AWEI_THRESH)
# ...AND are NOT snow.
is_water = is_awei_water.And(is_ice.Not())

# Combine into a final map
# 0=other, 1=snow, 2=water
classification = ee.Image(0).where(is_water, 2).where(is_ice, 1)

# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 13)

# True color (for comparison)
Map.addLayer(comp.clip(geom),
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3},
             'True Color')

# --- THIS IS THE FINAL CLASSIFICATION ---
# 0=transparent, 1=white (snow), 2=blue (water)
Map.addLayer(classification.clip(geom).selfMask(),
             {'min': 1, 'max': 2, 'palette':['white', 'blue']},
             'AWEI + NDSI (Ice=White, Water=Blue)')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary', True, 0.5)
display(Map)

Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

# Using your date
year, month = 2020, 8

start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Functions
# ========================================
def mask_clouds(img):
    """Masks clouds and cloud shadows in Sentinel-2 SCL."""
    scl = img.select('SCL')
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

def add_awei_ndsi(img):
    """Calculates AWEI (for water) and NDSI (for ice)."""

    bands = ['B2','B3','B4','B8','B11','B12']
    scaled = img.select(bands).multiply(0.0001)
    img = img.addBands(scaled, overwrite=True)

    # 1. AWEI (to find dark water)
    awei = img.expression(
        'B2 + 2.5*B3 - 1.5*(B8 + B11) - 0.25*B12',
        {'B2': img.select('B2'), 'B3': img.select('B3'),
         'B8': img.select('B8'), 'B11': img.select('B11'), 'B12': img.select('B12')}
    ).rename('AWEI')

    # 2. NDSI (to find snow/ice)
    ndsi = img.normalizedDifference(['B3','B11']).rename('NDSI')

    return img.addBands([awei, ndsi])

# ========================================
# Get composite
# ========================================
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start, end)
        .map(mask_clouds)
        .map(add_awei_ndsi))

comp = s2.median()

# ========================================
# Create the AWEI + NDSI Hybrid Classification
# ========================================
awei = comp.select('AWEI')
ndsi = comp.select('NDSI')

# --- THRESHOLDS TO TUNE ---
# Use the Inspector tool to find the real values for your lake
AWEI_THRESH = 0.4
NDSI_THRESH = 0.8
# ---------------------------

# 1. Class 1: SNOW/ICE
is_ice = ndsi.gt(NDSI_THRESH)

# 2. Class 2: WATER
is_awei_water = awei.gt(AWEI_THRESH)
is_water = is_awei_water.And(is_ice.Not())

# Combine into a final map
classification = ee.Image(0).where(is_water, 2).where(is_ice, 1)

# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 13)

# --- NEW LAYERS FOR INSPECTION ---
# Use the "Inspector" tool on these layers
Map.addLayer(ndsi.clip(geom),
             {'min': -1, 'max': 1, 'palette': ['white', 'black']},
             'Raw NDSI (for Inspector)')

Map.addLayer(awei.clip(geom),
             {'min': -0.5, 'max': 0.5, 'palette': ['red', 'white', 'blue']},
             'Raw AWEI (for Inspector)')
# ------------------------------------

# True color (for comparison)
Map.addLayer(comp.clip(geom),
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3},
             'True Color')

# Final Classification (based on your guessed thresholds)
Map.addLayer(classification.clip(geom).selfMask(),
             {'min': 1, 'max': 2, 'palette':['white', 'blue']},
             'Final Classification')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary', True, 0.5)
display(Map)

Map(center=[65.43821886646404, -50.20955628185561], controls=(WidgetControl(options=['position', 'transparent_…

In [ ]:
from skimage.filters import threshold_otsu
    # ... then use threshold_otsu on your image data

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

# Using your date
year, month = 2020, 9
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Functions
# ========================================
def mask_clouds(img):
    """Masks clouds and cloud shadows in Sentinel-2 SCL."""
    scl = img.select('SCL')
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

def add_awei_ndsi(img):
    """Calculates AWEI (for water) and NDSI (for ice)."""

    bands = ['B2','B3','B4','B8','B11','B12']
    scaled = img.select(bands).multiply(0.0001)
    img = img.addBands(scaled, overwrite=True)

    # 1. AWEI (to find dark water)
    awei = img.expression(
        'B2 + 2.5*B3 - 1.5*(B8 + B11) - 0.25*B12',
        {'B2': img.select('B2'), 'B3': img.select('B3'),
         'B8': img.select('B8'), 'B11': img.select('B11'), 'B12': img.select('B12')}
    ).rename('AWEI')

    # 2. NDSI (to find snow/ice)
    ndsi = img.normalizedDifference(['B3','B11']).rename('NDSI')

    return img.addBands([awei, ndsi])

# ========================================
# Get composite
# ========================================
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start, end)
        .map(mask_clouds)
        .map(add_awei_ndsi))

comp = s2.median()

awei = comp.select('AWEI')
ndsi = comp.select('NDSI')

# ========================================
# --- NEW: Automatic Threshold Calculation ---
# ========================================
# This reducer finds the optimal threshold to separate
# the two classes (e.g., water/ice) in the image
otsu_reducer = ee.Reducer.otsu()

# 1. Calculate the NDSI threshold
otsu_ndsi_dict = ndsi.reduceRegion(
    reducer=otsu_reducer,
    geometry=geom,
    scale=10,
    maxPixels=1e9
)
auto_ndsi_thresh = ee.Number(otsu_ndsi_dict.get('threshold'))

# 2. Calculate the AWEI threshold
otsu_awei_dict = awei.reduceRegion(
    reducer=otsu_reducer,
    geometry=geom,
    scale=10,
    maxPixels=1e9
)
auto_awei_thresh = ee.Number(otsu_awei_dict.get('threshold'))

# --- PRINT THE RESULTS ---
# Use .getInfo() to pull the server-side value
print('--- Automatic Thresholds (Otsu\'s Method) ---')
print('Calculated AWEI Threshold:', auto_awei_thresh.getInfo())
print('Calculated NDSI Threshold:', auto_ndsi_thresh.getInfo())
print('---------------------------------------------')


# ========================================
# Create the Hybrid Classification
# ========================================
# Use the new, automatically-calculated thresholds
AWEI_THRESH = auto_awei_thresh
NDSI_THRESH = auto_ndsi_thresh

# 1. Class 1: SNOW/ICE
is_ice = ndsi.gt(NDSI_THRESH)

# 2. Class 2: WATER
is_awei_water = awei.gt(AWEI_THRESH)
is_water = is_awei_water.And(is_ice.Not())

# Combine into a final map
classification = ee.Image(0).where(is_water, 2).where(is_ice, 1)

# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 13)

# True color (for comparison)
Map.addLayer(comp.clip(geom),
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3},
             'True Color')

# Final Classification (using automatic thresholds)
Map.addLayer(classification.clip(geom).selfMask(),
             {'min': 1, 'max': 2, 'palette':['white', 'blue']},
             'Auto-Threshold Classification')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary', True, 0.5)
display(Map)

AttributeError: type object 'Reducer' has no attribute 'otsu'

In [ ]:
import ee
import geemap
from IPython.display import display

ee.Initialize(project='menu-453815')

lakes = ee.FeatureCollection('users/ansh09feb/Lakes')
lake = lakes.first()
geom = lake.geometry()

# Using your date
year, month = 2020, 9
start = ee.Date.fromYMD(year, month, 1)
end = start.advance(1, 'month')

# ========================================
# Functions
# ========================================
def mask_clouds(img):
    """Masks clouds and cloud shadows in Sentinel-2 SCL."""
    scl = img.select('SCL')
    return img.updateMask(scl.neq(3).And(scl.neq(8)).And(scl.neq(9)).And(scl.neq(10)))

def add_nir(img):
    """Scales and adds only the NIR (B8) and visible bands."""

    bands = ['B2','B3','B4','B8']
    scaled = img.select(bands).multiply(0.0001)
    img = img.addBands(scaled, overwrite=True)

    # Get the NIR band (B8)
    nir = img.select('B8').rename('NIR')

    return img.addBands(nir)

# ========================================
# Get composite
# ========================================
s2 = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(geom)
        .filterDate(start, end)
        .map(mask_clouds)
        .map(add_nir))

comp = s2.median()

nir = comp.select('NIR')

# ========================================
# --- NEW: Automatic Threshold on NIR ---
# ========================================
# This reducer will find the valley between the
# "dark water" peak and the "bright ice" peak.
otsu_reducer = ee.Reducer.otsu()

otsu_nir_dict = nir.reduceRegion(
    reducer=otsu_reducer,
    geometry=geom,
    scale=10,
    maxPixels=1e9
)
auto_nir_thresh = ee.Number(otsu_nir_dict.get('threshold'))

# --- PRINT THE RESULTS ---
print('--- Automatic NIR Threshold (Otsu\'s Method) ---')
print('Calculated NIR Threshold:', auto_nir_thresh.getInfo())
print('(Pixels darker than this are water)')
print('------------------------------------------------')

# ========================================
# Create the Hybrid Classification
# ========================================
NIR_THRESH = auto_nir_thresh

# Class 2: WATER
# Pixels that are DARKER than the threshold
is_water = nir.lt(NIR_THRESH)

# Class 1: SNOW/ICE
# Pixels that are BRIGHTER than the threshold
is_ice = nir.gte(NIR_THRESH)

# Combine into a final map
classification = ee.Image(0).where(is_water, 2).where(is_ice, 1)

# ========================================
# VISUALIZE! 🎨
# ========================================
Map = geemap.Map()
Map.centerObject(geom, 13)

# True color (for comparison)
Map.addLayer(comp.clip(geom),
             {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.3},
             'True Color')

# Add the raw NIR band so you can inspect it
Map.addLayer(nir.clip(geom),
             {'min': 0, 'max': 0.5, 'palette': ['black', 'white']},
             'Raw NIR (B8)')

# Final Classification (using automatic thresholds)
Map.addLayer(classification.clip(geom).selfMask(),
             {'min': 1, 'max': 2, 'palette':['white', 'blue']},
             'NIR Auto-Threshold (Ice=White, Water=Blue)')

Map.addLayer(geom, {'color':'red'}, 'Lake boundary', True, 0.5)
display(Map)

AttributeError: type object 'Reducer' has no attribute 'otsu'